# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

## Step2 加载数据

In [2]:
import pandas as pd

data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


In [3]:
data = data.dropna()
data

,label,review
0,1,"距离川沙公路较近,但是公交指示不对,如果是""蔡陆线""的话,会非常麻烦.建议用别的路线.房间较..."
1,1,商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!
2,1,早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。
3,1,宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小...
4,1,"CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风"
...,...,...
7761,0,尼斯酒店的几大特点：噪音大、环境差、配置低、服务效率低。如：1、隔壁歌厅的声音闹至午夜3点许...
7762,0,盐城来了很多次，第一次住盐阜宾馆，我的确很失望整个墙壁黑咕隆咚的，好像被烟熏过一样家具非常的...
7763,0,看照片觉得还挺不错的，又是4星级的，但入住以后除了后悔没有别的，房间挺大但空空的，早餐是有但...
7764,0,我们去盐城的时候那里的最低气温只有4度，晚上冷得要死，居然还不开空调，投诉到酒店客房部，得到...


## Step3 创建Dataset

In [4]:
from torch.utils.data import Dataset

class MyDataset(Dataset):

    def __init__(self) -> None:
        super().__init__()
        self.data = pd.read_csv("./ChnSentiCorp_htl_all.csv")
        self.data = self.data.dropna()

    def __getitem__(self, index):
        return self.data.iloc[index]["review"], self.data.iloc[index]["label"]
    
    def __len__(self):
        return len(self.data)

In [5]:
dataset = MyDataset()
for i in range(5):
    print(dataset[i])

('距离川沙公路较近,但是公交指示不对,如果是"蔡陆线"的话,会非常麻烦.建议用别的路线.房间较为简单.', 1)
('商务大床房，房间很大，床有2M宽，整体感觉经济实惠不错!', 1)
('早餐太差，无论去多少人，那边也不加食品的。酒店应该重视一下这个问题了。房间本身很好。', 1)
('宾馆在小街道上，不大好找，但还好北京热心同胞很多~宾馆设施跟介绍的差不多，房间很小，确实挺小，但加上低价位因素，还是无超所值的；环境不错，就在小胡同内，安静整洁，暖气好足-_-||。。。呵还有一大优势就是从宾馆出发，步行不到十分钟就可以到梅兰芳故居等等，京味小胡同，北海距离好近呢。总之，不错。推荐给节约消费的自助游朋友~比较划算，附近特色小吃很多~', 1)
('CBD中心,周围没什么店铺,说5星有点勉强.不知道为什么卫生间没有电吹风', 1)


## Step4 划分数据集

In [6]:
from torch.utils.data import random_split


trainset, validset = random_split(dataset, lengths=[0.9, 0.1])
len(trainset), len(validset)

(6989, 776)

In [7]:
for i in range(10):
    print(trainset[i])

('这里环境还算不错,吃海鲜也便宜,当时买当时煮,很鲜', 1)
('酒店和房间的基本设施都还行,就是服务还需跟上', 1)
('刚从九洲环宇换到这个酒店，感觉差距太大了，这里号称在市中心，但周围除了天宁寺，没别的。房间有点旧了，老式格局，洗澡水很小。价钱比九洲环宇要高30%多，觉得非常非常不值。再也不会选择这里了，除非价钱降到200以下。', 0)
('朝阳的房间不算大，朝南的房间比较大，有大沙发，但是比较昏暗。大堂比较暗，不敞亮。位置比较靠中心，就是门口有单行。', 1)
('房价偏贵，设施旧，服务还可以，交通比较方便，携程比自己前台登记也没有优势', 1)
('服务太不好了，晚上房间的蚊子太多了，人性的关怀不好', 0)
('酒店总体来说还好。房间比较宽敞，但是说是4星，大概应该只有3星。没有一次性拖鞋比较郁闷。早餐不错，比较满意。交通非常方便，但是酒店外面的出租车最好别坐，太黑人。我们被拉到一家超级无敌宰客的地方吃海鲜，晕啊，三叠菜，500块，滴血。。。出门旅游，捂好荷包啊！', 1)
('一家不错的特色度假酒店，到处充满的茶文化，什么茶壶，茶经琳琅满目，估计现在是旅游的旺季，前两个星期就开始预订酒店了。但周末一直被告知订满了，也不知道是真是假，终于搞订。周末带着老婆出发。刚进酒店才就发现酒店生意真的是很好，总台办理入住的人太多，等了好久才轮到，在总台办理入住时，总台的服务员很热情，态度也不错。后被告知之酒店现在周末预订的话最好是提前二周时间预订。终于入住房间。房间不是很大，但是个大床间，床是大的可以了，估计有二米，卫生间小了一点，但配置还是蛮全的。房间整体还是很整洁的舒适的，茶几上还有水果，这到是个惊喜。看了房间里的说明才了解，陆羽山庄是中国首家以茶文化为主题的度假酒店。怪不得连房间里也布置了茶壶呢。下午去酒店边上的双溪漂流玩了一下，还不错就是人多的可以。回酒店后去了游泳池，酒店的泳池真是的不错。是室内外的，边上的环境也不错身边到处是绿色的世界，游完泳又玩了酒店的沙弧球、台球，没想到酒店不大康乐配备还是挺全的。晚上准备用餐，走过大堂，遇见酒店早上办理入住的大堂副理，听说我们要去用餐了，介绍酒店的中餐特色是以土菜新做，而且也是很实惠的。看在酒店服务这么殷切的份上和老婆商量去试一下。到餐厅后，发现餐厅是以陆羽的名字命名的，菜肴很有特色，服务员介绍了，小溪渔，特

## Step5 创建Dataloader

In [7]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def collate_func(batch):
    texts, labels = [], []
    for item in batch:
        texts.append(item[0])
        labels.append(item[1])
    inputs = tokenizer(texts, max_length=128, padding="max_length", truncation=True, return_tensors="pt")
    inputs["labels"] = torch.tensor(labels)
    return inputs

C:\Users\wdx\.conda\envs\wdx_torch\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [8]:
from torch.utils.data import DataLoader

trainloader = DataLoader(trainset, batch_size=32, shuffle=True, collate_fn=collate_func)
validloader = DataLoader(validset, batch_size=64, shuffle=False, collate_fn=collate_func)

In [9]:
next(enumerate(validloader))[1]

{'input_ids': tensor([[ 101, 4384, 1862,  ...,    0,    0,    0],
        [ 101, 6821, 3221,  ...,    0,    0,    0],
        [ 101, 1728,  711,  ...,    0,    0,    0],
        ...,
        [ 101, 7478, 2382,  ..., 6589, 2533,  102],
        [ 101, 6983, 2421,  ...,    0,    0,    0],
        [ 101,  679, 3221,  ...,    0,    0,    0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
        0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1,
        0, 1, 1

## Step6 创建模型及优化器

In [10]:
from torch.optim import Adam

model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

if torch.cuda.is_available():
    model = model.cuda()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
optimizer = Adam(model.parameters(), lr=2e-5)

## Step7 训练与验证

In [12]:
def evaluate():
    model.eval()
    acc_num = 0
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            acc_num += (pred.long() == batch["labels"].long()).float().sum()
    return acc_num / len(validset)

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        acc = evaluate()
        print(f"ep: {ep}, acc: {acc}")

## Step8 模型训练

In [13]:
train()

C:\Users\wdx\.conda\envs\wdx_torch\Lib\site-packages\transformers\models\bert\modeling_bert.py:439: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


ep: 0, global_step: 0, loss: 0.6366204619407654
ep: 0, global_step: 100, loss: 0.20524942874908447
ep: 0, global_step: 200, loss: 0.3390590250492096
ep: 0, acc: 0.8891752362251282


KeyboardInterrupt: 

## Step9 模型预测

In [14]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [15]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [16]:
pipe(sen)

[{'label': '好评！', 'score': 0.9922906756401062}]

In [20]:
pipe("You are a very good student.")

[{'label': '好评！', 'score': 0.7209451794624329}]